In [1]:
import pandas as pd

In [43]:
toto = pd.read_csv('Data/scrapers/Toto/totoAllSports.csv', index_col=0)
kambi = pd.read_csv('Data/scrapers/unibet/unibet_allSports_odds.csv', index_col=0)

In [44]:
# event name is key
# event keys: Wedstrijd = Wedstrijd, outcome = 1 vs 2
#             Over/Under criterion_label + line : 'Totaal Aantal Gewonnen Games door Alcaraz, Carlos' + 125000.0 vs Market Name: 'Carlos Alcaraz Aantal Games - Over/Under 12.5'
set(kambi[kambi['sport'] == 'TENNIS'][['criterion_label',
       'criterion_english_label', 'occurrence_type', 'lifetime',
       'bet_offer_type_id', 'bet_offer_type_name',
       'bet_offer_type_english_name', 'event_id', 'outcome_id',
       'outcome_label', 'outcome_english_label', 'odds', 'line', 'participant',
       'type', 'status',
       'cash_out_status', 'home_score', 'away_score', 'event_name']].criterion_label)

{'Alex Minaur Wint een Set',
 'Alexander Zverev Wint een Set',
 'Andrey Rublev Wint een Set',
 'Carlos Alcaraz Wint een Set',
 'Casper Ruud Wint een Set',
 'Correcte Score - Set 1',
 'Daniil Medvedev Wint een Set',
 'Game Handicap',
 'Game Handicap - Set 1',
 'Jannik Sinner Wint een Set',
 'Meeste aces',
 'Set Handicap',
 'Setwedden',
 'Taylor Fritz Wint een Set',
 'Totaal Aantal Games',
 'Totaal Aantal Games - Set 1',
 'Totaal Aantal Games - Set 2',
 'Totaal Aantal Gewonnen Games door Alcaraz, Carlos',
 'Totaal Aantal Gewonnen Games door De Minaur, Alex',
 'Totaal Aantal Gewonnen Games door Fritz, Taylor',
 'Totaal Aantal Gewonnen Games door Medvedev, Daniil',
 'Totaal Aantal Gewonnen Games door Rublev, Andrey',
 'Totaal Aantal Gewonnen Games door Ruud, Casper',
 'Totaal Aantal Gewonnen Games door Sinner, Jannik',
 'Totaal Aantal Gewonnen Games door Zverev, Alexander',
 'Totaal Aantal Sets',
 'Totaal Aantal Tiebreaks',
 'Totaal aantal minuten',
 'Totaal aantal servicebreaks',
 'Wedstr

In [40]:
kambi[kambi['criterion_label'].str.contains('Totaal Aantal Gewonnen Games door Medvedev, Daniil')][['criterion_label',
       'criterion_english_label', 'occurrence_type', 'lifetime',
       'bet_offer_type_id', 'bet_offer_type_name',
       'bet_offer_type_english_name', 'event_id', 'outcome_id',
       'outcome_label', 'outcome_english_label', 'odds', 'line', 'participant',
       'type', 'status',
       'cash_out_status', 'home_score', 'away_score', 'event_name']].tail(60)

,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,status,cash_out_status,home_score,away_score,event_name
48,"Totaal Aantal Gewonnen Games door Medvedev, Da...","Total games won by Medvedev, Daniil",NaN,NaN,6,Over/Onder,Over/Under,1022071901,3565589032,Meer dan,Over,1890.0,12500.0,NaN,OT_OVER,OPEN,ENABLED,NaN,NaN,Daniil Medvedev vs De Alex Minaur
49,"Totaal Aantal Gewonnen Games door Medvedev, Da...","Total games won by Medvedev, Daniil",NaN,NaN,6,Over/Onder,Over/Under,1022071901,3565589033,Minder dan,Under,1910.0,12500.0,NaN,OT_UNDER,OPEN,ENABLED,NaN,NaN,Daniil Medvedev vs De Alex Minaur


In [32]:
set(toto[(toto['sport']=='Tennis') & (toto['event_id']==5943214)].drop_duplicates()['Market Name'])

{'1e Set - Wedstrijd Resultaat',
 'Aantal Games - Odd/Even',
 'Aantal Games - Over/Under 21.5',
 'Aantal Games - Over/Under 22.5',
 'Aantal Games - Over/Under 23.5',
 'Alex De Minaur Aantal Games - Over/Under 11.5',
 'Alex De Minaur Aantal Games - Over/Under 12.5',
 'Alex De Minaur Aantal Games - Over/Under 13.5',
 'Alex De Minaur Wint een Set',
 'Any Set Win To Nil',
 'Daniil Medvedev Aantal Games - Over/Under 11.5',
 'Daniil Medvedev Aantal Games - Over/Under 12.5',
 'Daniil Medvedev Aantal Games - Over/Under 13.5',
 'Daniil Medvedev Wint een Set',
 'Exact Sets',
 'Game - Handicap Winnaar -0.5',
 'Game - Handicap Winnaar -1.5',
 'Game - Handicap Winnaar 0.5',
 'Match Specials',
 'Match Winner and Total Games Over/Under 22.5',
 'Price Boost',
 'Set - Correct Score',
 'Set - Handicap -1.5',
 'Set - Handicap 1.5',
 'Set 1 Aantal Games - Over/Under 10.5',
 'Set 1 Aantal Games - Over/Under 8.5',
 'Set 1 Aantal Games - Over/Under 9.5',
 'Set 1 Correct Score',
 'Set 1 Game - Handicap Winnaa